## Week 5: Aggregating Data

Today you will learn how to aggregate data using R. By *aggregate*, I mean that we will be grouping observations on one variable, and organizing the remaining variables using summary statistics like mean, frequency, maximum, or minimum. This is a very common task, and it's not difficult to do thanks to [Hadley Wickam's]() package, [dplyr](https://github.com/hadley/dplyr). Dplyr provides a "grammar of data manipulation" and we will now take a look at its most useful components.

In [1]:
set.seed(193)

# Always load plyr first
library(plyr)
library(dplyr)
#install.packages("nycflights13")
library(nycflights13)

# Let's take a peek at the dataset
dim(flights)
head(flights)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



[1] 336776     19

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
2013,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,2013-01-01 05:00:00
2013,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,2013-01-01 05:00:00
2013,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,2013-01-01 05:00:00
2013,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45,2013-01-01 05:00:00
2013,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,2013-01-01 06:00:00
2013,1,1,554,558,-4,740,728,12,UA,1696,N39463,EWR,ORD,150,719,5,58,2013-01-01 05:00:00


## Basic dplyr functions

Some of the helpful functions in the package are also the most simple.
- <code>filter()</code> filters the rows based on conditions
- <code>arrange()</code> reorders rows 
    - Orders ascending by default, we can use <code>desc()</code> to change that
- <code>select()</code> filters columns
- <code>distinct()</code> finds distinct values in a variable
- <code>mutate()</code> creates new columns
    - Use <code>transmute()</code> to keep only the new columns
- <code>summarise()</code> collapses rows using a summary statistics
    - Uses functions like <code>mean()</code>, <code>max()</code>, and <code>n()</code>

You may notice that last lecture we learned how to do some of these things, like filtering rows or making new columns, using native R syntax. When we get to dplyr's piping operator, <code>%>%</code>, you will need to use these dplyr functions.

In [2]:
# Filter()'s basic syntax...
# newData <- filter(oldData, condition1, condition2, ..., conditionN)

allJanuary <- filter(flights, month ==1)
janAndAA <- filter(flights, month == 1, carrier == 'AA')

# Arrange()'s basic syntax...
# newData <- arrange(oldData, column1, column2, ..., columnN)

longFlights <- arrange(flights, distance)
doubleLong <- arrange(flights, distance, air_time) # This means we arrange by distance, and break ties by air_time

# Mutate's basic syntax...
# newData <- arrange(oldData, newColumn = [operation on columns], newColumn2 = [another operation], ...)

simpleLogical <- mutate(flights, 
                       longFlight = distance > mean(distance))
justInteraction <- transmute(flights,
                     lengthXtime = distance*air_time)

# Summarise's basic syntax...
# newData <- arrange(oldData, summaryColumn = [operation on columns], summaryCol2 = [operation on cols], ...)

simpleSummary <- summarise(flights, 
                          meanDelay = mean(arr_delay, na.rm = T),
                          maxDistance = max(distance),
                          count = n())

In [3]:
# Uncomment any dataframes you're interested in checking
# head(allJanuary)
# head(janAndAA)
# head(longFlights)
# head(doubleLong)
# head(simpleLogical)
# head(justInteraction)
# head(simpleSummary)

# There should only be one distinct month in allJanuary
# and only one carrier in janAndAA

distinct(allJanuary, month)
distinct(janAndAA, carrier)

month
1


carrier
AA


## Grouping

The above functions are great on their own, but they really shine when combined with <code>group_by()</code>. The function takes a dataframe as its first argument, followed by a list of columns to group by, and returns a new dataframe. Then, we can apply the previous functions and they will act on the grouping variables.

Let's work through an example! Say we're interested in the distribution of delay times among the carriers in the dataset. So we will need to group on <code>carrier</code> and generate some summary statistics from <code>arr_delay</code> and <code>dep_delay</code>.

In [4]:
# Group on carrier
carrierGroup <- group_by(flights, carrier)

# Get mean of the arrival and departure delays

## -- MAKE THIS A <FILL-IN> EXERCISE --
too_many_stats <- summarise(carrierGroup,
                         mean_dep = mean(dep_delay, na.rm=T),
                         mean_arr = mean(arr_delay, na.rm=T),
                         max_dep = max(dep_delay),
                         max_arr = max(arr_delay),
                         count = n())

# Select the relevant columns
summary_stats <- select(too_many_stats, carrier, mean_dep, 
                        mean_arr, max_dep, max_arr, count)

# Uncomment the other commands to see the intermediate steps
#head(carrierGroup)
#head(too_many_stats)
head(summary_stats)

carrier,mean_dep,mean_arr,max_dep,max_arr,count
9E,16.725769,7.3796692,NA,NA,18460
AA,8.586016,0.3642909,NA,NA,32729
AS,5.804775,-9.9308886,NA,NA,714
B6,13.022522,9.4579733,NA,NA,54635
DL,9.264505,1.6443409,NA,NA,48110
EV,19.955390,15.7964311,NA,NA,54173


## The piping operator, %>%

*Now let's dive into dplyr's real power.* As you saw from the last exercise, dplyr is very useful when we chain together the basic functions with <code>group_by()</code>. However, the process in the last code chunk was rather cumbersome. There were a lot of unnecessary dataframes, so let's use the piping operator: <code>%>%</code>. This operator will chain our inputs to subsequent functions. Put programatically, it takes <code>f(x, y)</code> and turns it into <code>x %>% f(y)</code>.

Let's see the previous code chunk done with piping. Notice that we can actually drop the <code>select()</code> statement.

In [5]:
usingPipes <- flights %>%
            group_by(carrier) %>%
            summarise(mean_dep = mean(dep_delay, na.rm=T),
                      mean_arr = mean(arr_delay, na.rm=T),
                      max_dep = max(dep_delay, na.rm=T),
                      max_arr = max(arr_delay, na.rm=T),
                      count = n())
head(usingPipes)

carrier,mean_dep,mean_arr,max_dep,max_arr,count
9E,16.725769,7.3796692,747,744,18460
AA,8.586016,0.3642909,1014,1007,32729
AS,5.804775,-9.9308886,225,198,714
B6,13.022522,9.4579733,502,497,54635
DL,9.264505,1.6443409,960,931,48110
EV,19.955390,15.7964311,548,577,54173


Lastly, let's investigate trends of delays by the time of day. We can use <code>mutate()</code> to quickly bin the hours of the day.

In [6]:
# Nested ifelse() functions act as normal if/else statements
# Basic syntax: ifelse(condition, if_True, if_False)
flights2 <- mutate(flights, 
                   quarterOfDay = ifelse(hour >= 0 & hour < 6, 0, 
                                         ifelse(hour >= 6 & hour < 12, 1, 
                                                ifelse(hour >= 12 & hour < 18, 2, 3))))

delaysByDay <- flights2 %>%
                group_by(quarterOfDay) %>%
                summarise(count = n(),
                          dep_longest = max(dep_delay, na.rm=T),
                          arr_longest = max(arr_delay, na.rm=T),
                          mean_dep = mean(dep_delay, na.rm=T),
                          mean_arr = mean(arr_delay, na.rm=T))
head(delaysByDay)

quarterOfDay,count,dep_longest,arr_longest,mean_dep,mean_arr
0,1954,201,208,0.6877572,-4.796907
1,129067,1301,1272,3.9934566,-1.777916
2,131159,1126,1109,15.5022961,10.427029
3,74596,1137,1127,23.1408445,16.269685


Looks like flights in the early AM have the shortest delays, remember that when you start planning your own vactions! This could be caused by the lower amount of flights overall (looking at the <code>count</code> variable).

## Joining Dataframes

Very quickly, I would like to cover how to merge dataframes in R. This is also a common task if you end up using R in your career. Imagine you have a dataframe of your customers' transaction histories and a second dataframe with your customers' account balances. At some point, you'll probably need to merge them. <strong>There are four basic types of joins</strong> and they all require a common column between them, called the ID variable. 

- <strong>Outter Join</strong>: When joining two dataframes A and B, this type of merge will keep all observations from both dataframes. If there is a record in A that does not have a matching id in B, it will still be kept (the reverse is also true).
- <strong>Inner Join</strong>: When joining A and B, this merge will only keep records for which there is a match on the id in <em>both</em> A and B.
- <strong>Left Join</strong>: When joining A and B, this merge will keep all records in A (because it's on the <em>left</em>) and only keep records from B for which there is a match on the id variable.
- <strong>Right Join</strong>: When joining A and B, this merge will keep all records in B (because it's on the <em>right</em>) and only keep records from A for which there is a match on the id variable.

This is a common task in [SQL](https://en.wikipedia.org/wiki/SQL), and some people make a living writing very complex SQL queries! While the basics of SQL are simple, joining and filtering database tables can become a difficult task--with queries ranging from 30 to 50 lines long. You can begin learning the basics of SQL queries right in RStudio by using Gabor Grothendieck's [sqldf package](https://cran.r-project.org/web/packages/sqldf/sqldf.pdf).

The two dataframes below have 40 transactions from our 10 customers. Looking at the 

In [7]:
# Transaction history - 40 rows
transactions <- data.frame(customerId=sample(1:10, 40, replace=T),
                           transAmount=round(runif(n=40, min=10, max=75),2),
                           nGoods=sample(1:10, size=40, replace=T)
                          )

# Customer data - 10 rows
customers <- data.frame(customerId=seq(1,10),
                        customerSince=sample(1990:2016, size=10, replace=T),
                        age=round(runif(n=10, min=30, max=65),0),
                        sex=sample(c(0,1), size=10, replace=T)
                       )

head(transactions)
head(customers)

customerId,transAmount,nGoods
6,26.43,6
2,66.28,3
4,59.82,1
2,61.70,9
9,30.26,7
6,59.42,9


customerId,customerSince,age,sex
1,2007,50,1
2,1999,54,0
3,2008,65,0
4,1992,59,1
5,2014,46,1
6,1991,37,0


In [8]:
# Left join transactions and customers
leftJoin <- join(x=transactions, y=customers, 
               by="customerId", type="left")

# Right join transactions and customers
rightJoin <- join(x=transactions, y=customers, 
               by="customerId", type="right")

head(leftJoin)
head(rightJoin)

customerId,transAmount,nGoods,customerSince,age,sex
6,26.43,6,1991,37,0
2,66.28,3,1999,54,0
4,59.82,1,1992,59,1
2,61.70,9,1999,54,0
9,30.26,7,2004,63,0
6,59.42,9,1991,37,0


customerId,transAmount,nGoods,customerSince,age,sex
1,29.04,2,2007,50,1
1,63.77,3,2007,50,1
1,18.60,10,2007,50,1
2,66.28,3,1999,54,0
2,61.70,9,1999,54,0
2,16.73,5,1999,54,0


## Exercises

As always, these exercises will start easy and slowly get more difficult.

In [2]:
# let's use a dataframe from the ggplot2 package
# you will need to install the package if you have not already

#install.packages("ggplot2")

library(ggplot2)
head(diamonds)

carat,cut,color,clarity,depth,table,price,x,y,z
0.23,Ideal,E,SI2,61.5,55,326,3.95,3.98,2.43
0.21,Premium,E,SI1,59.8,61,326,3.89,3.84,2.31
0.23,Good,E,VS1,56.9,65,327,4.05,4.07,2.31
0.29,Premium,I,VS2,62.4,58,334,4.20,4.23,2.63
0.31,Good,J,SI2,63.3,58,335,4.34,4.35,2.75
0.24,Very Good,J,VVS2,62.8,57,336,3.94,3.96,2.48


In [19]:
# Order the dataframe by the price with the highest prices at the top
# Break any ties in price by the carat of the diamond

# Do NOT using the piping operator

#orderedPrice <- <FILLIN>
orderedPrice <- arrange(diamonds, desc(price), carat)

head(orderedPrice)
stopifnot(orderedPrice$price[1] == 18823)

carat,cut,color,clarity,depth,table,price,x,y,z
2.29,Premium,I,VS2,60.8,60,18823,8.50,8.47,5.16
2.00,Very Good,G,SI1,63.5,56,18818,7.90,7.97,5.04
1.51,Ideal,G,IF,61.7,55,18806,7.37,7.41,4.56
2.07,Ideal,G,SI2,62.5,55,18804,8.20,8.13,5.11
2.00,Very Good,H,SI1,62.8,57,18803,7.95,8.00,5.01
2.29,Premium,I,SI1,61.8,59,18797,8.52,8.45,5.24


In [21]:
# Now accomplish the same task using the piping operator

#orderedPrice2 <- <FILLIN> %>%
#                 <FILLIN>

orderedPrice2 <- diamonds %>%
                arrange(desc(price), carat)

stopifnot(orderedPrice2$price[2] == 18818)

In [10]:
# The variables "x", "y", and "z" are meaurements on the length, width, and height.
# Use mutate to create a new column called "volume", this new variable should
# equal the volume of the diamond. Round the values to two decimal points.

#diamonds <- mutate(<FILLIN>)
diamonds <- mutate(diamonds, volume = round(x*y*z, 2))
head(diamonds)

stopifnot(diamonds$volume[1] == 38.20)

carat,cut,color,clarity,depth,table,price,x,y,z,overMean,volume
0.23,Ideal,E,SI2,61.5,55,326,3.95,3.98,2.43,0,38.20
0.21,Premium,E,SI1,59.8,61,326,3.89,3.84,2.31,0,34.51
0.23,Good,E,VS1,56.9,65,327,4.05,4.07,2.31,0,38.08
0.29,Premium,I,VS2,62.4,58,334,4.20,4.23,2.63,0,46.72
0.31,Good,J,SI2,63.3,58,335,4.34,4.35,2.75,0,51.92
0.24,Very Good,J,VVS2,62.8,57,336,3.94,3.96,2.48,0,38.69


In [11]:
# use mutate() to create a new logical column called "overMean"
# This variable should equal 1 if the price is over the mean price,
# and be 0 if the price is under the mean

#diamonds <- mutate(<FILLIN>)
diamonds <- mutate(diamonds, overMean = ifelse(price > mean(price),1,0))


head(diamonds)
stopifnot(diamonds$overMean[1] == 0 && diamonds$overMean[nrow(diamonds)] == 0)
# Hint: What about a UDF? Or maybe ifelse()?

carat,cut,color,clarity,depth,table,price,x,y,z,overMean,volume
0.23,Ideal,E,SI2,61.5,55,326,3.95,3.98,2.43,0,38.20
0.21,Premium,E,SI1,59.8,61,326,3.89,3.84,2.31,0,34.51
0.23,Good,E,VS1,56.9,65,327,4.05,4.07,2.31,0,38.08
0.29,Premium,I,VS2,62.4,58,334,4.20,4.23,2.63,0,46.72
0.31,Good,J,SI2,63.3,58,335,4.34,4.35,2.75,0,51.92
0.24,Very Good,J,VVS2,62.8,57,336,3.94,3.96,2.48,0,38.69


In [12]:
# Group the dataframe by the cut of the diamond and create
# the followng summary statistics...
# meanPrice ... the mean price for that group
# meanVol ... the average volume for that group
# maxCarat ... the maximum carat value
# minCarat ... the minimum carat value

# Use the piping operator
grouped1 <- diamonds %>%
            group_by(cut) %>%
            summarise(meanPrice = mean(price),
                      meanVol = mean(volume),
                      maxCarat = max(carat),
                      minCarat = min(carat))
head(grouped1)

stopifnot(grouped1$cut[1] == "Fair" & grouped1$minCarat[5] == 0.20)

cut,meanPrice,meanVol,maxCarat,minCarat
Fair,4358.758,164.9507,5.01,0.22
Good,3928.864,136.2573,3.01,0.23
Very Good,3981.760,130.9998,4.00,0.20
Premium,4584.258,145.0521,4.01,0.20
Ideal,3457.542,115.3949,3.50,0.20


In [17]:
# Do the same grouping again, you can copy/paste your solution,
# but add the following summary variables:
# n_D ... the number of diamonds with color="D" in each group
# n_J ... the number of diamonds with color="J" in each group

# To do this, you will need to write a helper function. It should take
# the column name and color name as arguments and return the number of elements
# in the column that equal that color.
helper <- function(column, color){
    return(sum(as.numeric(column == color)))
}

# Copy/paste your old solution here. 
# Then use your UDF to add the new columns.
grouped2 <- diamonds %>%
            group_by(cut) %>%
            summarise(meanPrice = mean(price),
                      meanVol = mean(volume),
                      maxCarat = max(carat),
                      minCarat = min(carat),
                      n_D = helper(color, "D"),
                      n_J = helper(color, "J"))
head(grouped2)

stopifnot(grouped2$n_D[1] == 163 & grouped2$n_J[1] == 119)

cut,meanPrice,meanVol,maxCarat,minCarat,n_D,n_J
Fair,4358.758,164.9507,5.01,0.22,163,119
Good,3928.864,136.2573,3.01,0.23,662,307
Very Good,3981.760,130.9998,4.00,0.20,1513,678
Premium,4584.258,145.0521,4.01,0.20,1603,808
Ideal,3457.542,115.3949,3.50,0.20,2834,896
